# Step 12 — Security Layer (PII Redaction + Request Signing)

**What we built**: A `SecurityModule` that performs two phases per `invoke()`: (1) redact PII from outbound messages and inbound responses, and (2) sign request payloads with HMAC-SHA256 for non-repudiation.

**Why it matters**: Federal environments (NIST 800-53) require PII protection and audit non-repudiation. Thousands of autonomous agents sending user data to LLMs need automatic PII scrubbing — agents shouldn't have to think about it. Request signing proves which model saw which data.

**Key decisions**:
- **D-089**: Two separate concerns — `VaultResolver` (construction-time key resolution) + `SecurityModule` (per-invoke PII + signing)
- **D-093**: Regex default + pluggable PII detector protocol. Built-in: SSN, credit card, email, phone, IPv4. Custom patterns addable via config.
- **D-094**: Type-tagged redaction: `[PII:SSN]`, `[PII:EMAIL]` — preserves type info for debugging
- **D-095**: HMAC-SHA256 default (stdlib, zero deps). ECDSA P-256 optional (`arcllm[signing]`)
- **D-096**: Sign messages + tools + model name — covers everything affecting LLM behavior
- **D-097**: Single `SecurityModule` with PII phase + signing phase in one `invoke()`
- **D-099**: Stack: `Audit → Security → Retry` (audit sees redacted data; each retry sends redacted+signed request)

**Stack position**: `Otel → Telemetry → Audit → Security → Retry → Fallback → RateLimit → Adapter`

**New components**: `_pii.py` (PII detection/redaction), `_signing.py` (canonical payload + HMAC signer)

In [ ]:
# Setup: ensure arcllm is importable
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

---
## 1. PII Detection — Built-in Patterns

The `RegexPiiDetector` ships with 5 built-in patterns that catch the most common PII types in federal contexts.

In [ ]:
from arcllm._pii import RegexPiiDetector, PiiMatch, redact_text, _BUILTIN_PATTERNS

print("Built-in PII patterns:")
for name, pattern in _BUILTIN_PATTERNS:
    print(f"  {name:15s} → {pattern.pattern}")

In [ ]:
detector = RegexPiiDetector()

# SSN detection
text = "Patient SSN: 123-45-6789"
matches = detector.detect(text)
print(f"Text:    {text}")
print(f"Matches: {matches}")
print(f"Redacted: {redact_text(text, matches)}")

In [ ]:
# Multiple PII types in one string
text = "Contact John at john@example.com or 555-123-4567. SSN: 987-65-4321."
matches = detector.detect(text)

print(f"Text: {text}")
print(f"\nMatches ({len(matches)}):")
for m in matches:
    print(f"  {m.pii_type:15s} [{m.start}:{m.end}] = {m.matched_text!r}")

print(f"\nRedacted: {redact_text(text, matches)}")

In [ ]:
# Credit card detection
text = "Card number: 4111-1111-1111-1111"
matches = detector.detect(text)
print(f"Text:     {text}")
print(f"Redacted: {redact_text(text, matches)}")

# IPv4 detection
text = "Server at 192.168.1.100 responded"
matches = detector.detect(text)
print(f"\nText:     {text}")
print(f"Redacted: {redact_text(text, matches)}")

---
## 2. Type-Tagged Redaction (D-094)

Replacements use `[PII:TYPE]` format — standard in federal systems. The type tag preserves debugging context without revealing the actual data.

In [ ]:
examples = [
    "SSN is 123-45-6789",
    "Email: alice@gov.mil",
    "Call 555-867-5309",
    "CC: 4111 1111 1111 1111",
    "IP: 10.0.0.1",
]

for text in examples:
    matches = detector.detect(text)
    redacted = redact_text(text, matches)
    print(f"  {text:40s} → {redacted}")

print("\nType tags let you grep for what KIND of PII was present")
print("without exposing the actual data.")

---
## 3. Overlap Handling

When PII patterns overlap (e.g., a phone number that looks like an SSN), the longer match wins.

In [ ]:
# No PII → empty matches, original text returned
text = "The weather is nice today."
matches = detector.detect(text)
print(f"No PII: matches={matches}, redacted={redact_text(text, matches)!r}")

# Empty string
matches = detector.detect("")
print(f"Empty:  matches={matches}")

---
## 4. Custom PII Patterns

The detector accepts custom regex patterns via constructor — for domain-specific PII (MRNs, badge numbers, etc.).

In [ ]:
# Add a custom pattern for Medical Record Numbers (MRN)
custom_detector = RegexPiiDetector(
    custom_patterns=[
        {"name": "MRN", "pattern": r"MRN-\d{6}"},
        {"name": "BADGE", "pattern": r"BADGE-[A-Z]{2}\d{4}"},
    ]
)

text = "Patient MRN-123456 treated by BADGE-AB1234. Email: doc@hospital.org"
matches = custom_detector.detect(text)

print(f"Text: {text}")
print(f"\nMatches ({len(matches)}):")
for m in matches:
    print(f"  {m.pii_type:15s} = {m.matched_text!r}")

print(f"\nRedacted: {redact_text(text, matches)}")

In [ ]:
from arcllm.exceptions import ArcLLMConfigError

# Invalid regex in custom pattern → clear error
try:
    RegexPiiDetector(custom_patterns=[{"name": "BAD", "pattern": r"["}])
except ArcLLMConfigError as e:
    print(f"Bad regex caught: {e}")

---
## 5. Pluggable Detector Protocol

The `PiiDetector` protocol allows replacing the regex engine with Presidio, spaCy, or any custom backend.

In [ ]:
from arcllm._pii import PiiDetector
import inspect

print(inspect.getsource(PiiDetector))
print("\nAny class implementing detect(text) -> list[PiiMatch] works.")
print("Use config: pii_detector='custom' + register your class.")

---
## 6. Request Signing — HMAC-SHA256

Request signing creates a deterministic signature of the request payload. This proves:
- **What** was sent to the LLM (messages + tools + model)
- **Integrity** — payload wasn't tampered with
- **Non-repudiation** — NIST AU-10 compliance

In [ ]:
from arcllm._signing import canonical_payload, HmacSigner
from arcllm.types import Message, Tool

messages = [
    Message(role="system", content="You are helpful."),
    Message(role="user", content="What is 2+2?"),
]
tools = [Tool(name="calc", description="Calculator", parameters={"type": "object"})]

# Step 1: Create canonical payload (deterministic JSON bytes)
payload = canonical_payload(messages, tools, "claude-sonnet-4-20250514")
print(f"Canonical payload ({len(payload)} bytes):")
print(f"  {payload[:120]}...")
print()

# Determinism: same input always produces same bytes
payload2 = canonical_payload(messages, tools, "claude-sonnet-4-20250514")
print(f"Deterministic: {payload == payload2}")

In [ ]:
# Step 2: Sign with HMAC-SHA256
signer = HmacSigner(key=b"my-secret-signing-key")

sig1 = signer.sign(payload)
sig2 = signer.sign(payload)
print(f"Signature: {sig1}")
print(f"Same input = same sig: {sig1 == sig2}")

# Different payload → different signature
other_payload = canonical_payload(
    [Message(role="user", content="different")], None, "gpt-4"
)
sig3 = signer.sign(other_payload)
print(f"Different payload:     {sig3}")
print(f"Signatures differ: {sig1 != sig3}")

In [ ]:
print(inspect.getsource(canonical_payload))
print("\nSort keys + compact separators = deterministic JSON.")
print("Same input ALWAYS produces same bytes, regardless of Python dict ordering.")

---
## 7. Signer Factory — Key from Environment Variable

In [ ]:
from arcllm._signing import create_signer

# Set the signing key in env
os.environ["ARCLLM_SIGNING_KEY"] = "my-secret-key"

signer = create_signer("hmac-sha256", "ARCLLM_SIGNING_KEY")
print(f"Signer type: {type(signer).__name__}")
print(f"Signature: {signer.sign(b'test')}")

In [ ]:
# Missing key → clear error
try:
    create_signer("hmac-sha256", "NONEXISTENT_KEY_VAR")
except ArcLLMConfigError as e:
    print(f"Missing key: {e}")

# Unsupported algorithm
try:
    create_signer("rsa-2048", "ARCLLM_SIGNING_KEY")
except ArcLLMConfigError as e:
    print(f"\nBad algorithm: {e}")

# ECDSA needs optional dep
try:
    create_signer("ecdsa-p256", "ARCLLM_SIGNING_KEY")
except ArcLLMConfigError as e:
    print(f"\nECDSA: {e}")

---
## 8. SecurityModule — Full invoke() Flow

The module runs four phases per `invoke()`:

```
1. Redact PII from outbound messages (to LLM)
2. Call inner.invoke() with redacted messages
3. Redact PII from inbound response (from LLM)
4. Sign request payload and attach signature to response metadata
```

In [ ]:
from unittest.mock import AsyncMock, MagicMock
from arcllm.modules.security import SecurityModule
from arcllm.types import LLMProvider, LLMResponse, Usage

os.environ["ARCLLM_SIGNING_KEY"] = "test-signing-key"

# Track what messages the inner adapter receives
captured_messages = []

async def capture_invoke(messages, tools=None, **kwargs):
    captured_messages.clear()
    captured_messages.extend(messages)
    return LLMResponse(
        content="Patient John Smith has diabetes.",  # Response also has PII
        usage=Usage(input_tokens=50, output_tokens=30, total_tokens=80),
        model="claude-sonnet-4-20250514",
        stop_reason="end_turn",
    )

inner = MagicMock(spec=LLMProvider)
inner.name = "anthropic"
inner.model_name = "claude-sonnet-4-20250514"
inner.invoke = capture_invoke

module = SecurityModule({}, inner)

# Messages with PII
messages = [
    Message(role="system", content="You are a medical assistant."),
    Message(role="user", content="Patient SSN: 123-45-6789, email: patient@hospital.org"),
]

result = await module.invoke(messages)

print("=== What the LLM received (after outbound redaction) ===")
for msg in captured_messages:
    print(f"  [{msg.role}] {msg.content}")

print(f"\n=== What the agent received (after inbound redaction) ===")
print(f"  Response: {result.content}")

print(f"\n=== Signature attached ===")
print(f"  Algorithm: {result.metadata.get('signing_algorithm')}")
print(f"  Signature: {result.metadata.get('request_signature')[:40]}...")

Notice:
1. **Outbound**: SSN and email replaced with `[PII:SSN]` and `[PII:EMAIL]` before the LLM sees them
2. **Inbound**: The LLM response is also scanned for PII (the LLM might echo back sensitive data)
3. **Signature**: Attached to `response.metadata` for audit trail verification

---
## 9. Content Block Redaction

PII redaction handles all content types — not just strings.

In [ ]:
from arcllm.types import TextBlock, ToolResultBlock, ToolUseBlock

captured_messages.clear()

# Messages with ContentBlock lists
messages_with_blocks = [
    Message(role="user", content=[
        TextBlock(text="Look up patient with SSN 111-22-3333"),
    ]),
    Message(role="tool", content=[
        ToolResultBlock(
            tool_use_id="call_1",
            content="Found: John Smith, email john@hospital.org, phone 555-123-4567"
        ),
    ]),
]

inner2 = MagicMock(spec=LLMProvider)
inner2.name = "anthropic"
inner2.model_name = "claude-sonnet-4-20250514"
inner2.invoke = capture_invoke

module2 = SecurityModule({}, inner2)
result = await module2.invoke(messages_with_blocks)

print("=== Redacted ContentBlocks sent to LLM ===")
for msg in captured_messages:
    if isinstance(msg.content, list):
        for block in msg.content:
            if isinstance(block, TextBlock):
                print(f"  TextBlock: {block.text}")
            elif isinstance(block, ToolResultBlock):
                print(f"  ToolResultBlock: {block.content}")
    else:
        print(f"  [{msg.role}] {msg.content}")

---
## 10. Config Options

In [ ]:
from arcllm.modules.security import _VALID_CONFIG_KEYS

print(f"Valid config keys: {sorted(_VALID_CONFIG_KEYS - {'enabled'})}")
print()

# PII only (no signing)
pii_only = SecurityModule({"signing_enabled": False}, inner)
print(f"PII only:     pii={pii_only._pii_enabled}, signing={pii_only._signing_enabled}")

# Signing only (no PII)
sign_only = SecurityModule({"pii_enabled": False}, inner)
print(f"Signing only: pii={sign_only._pii_enabled}, signing={sign_only._signing_enabled}")

# Custom patterns via config
custom = SecurityModule({
    "pii_custom_patterns": [{"name": "MRN", "pattern": r"MRN-\d{6}"}],
    "signing_key_env": "ARCLLM_SIGNING_KEY",
}, inner)
print(f"\nCustom patterns: {len(custom._pii_detector._patterns)} patterns (5 builtin + 1 custom)")

In [ ]:
# Config validation: unknown keys
try:
    SecurityModule({"pii_enbled": True}, inner)  # Typo
except ArcLLMConfigError as e:
    print(f"Typo caught: {e}")

# Invalid detector type
try:
    SecurityModule({"pii_detector": "spacy"}, inner)
except ArcLLMConfigError as e:
    print(f"\nBad detector: {e}")

---
## 11. Stack Position — Why Audit → Security → Retry?

```
Audit    ← sees redacted messages (PII-safe audit trail)
  └─ Security ← redacts PII, signs request
      └─ Retry ← each retry sends redacted+signed request
          └─ Adapter (HTTP call)
```

- **Audit sees redacted data**: The audit trail never contains raw PII
- **Each retry is redacted + signed**: If retry sends a new request, it's already clean
- **Signature covers redacted content**: The signature proves what the LLM actually saw (redacted version)

In [ ]:
from arcllm.registry import load_model, clear_cache

os.environ.setdefault("ANTHROPIC_API_KEY", "test-key")
clear_cache()

model = load_model(
    "anthropic",
    audit=True,
    security=True,
    retry=True,
)

layers = []
layer = model
while hasattr(layer, '_inner'):
    layers.append(type(layer).__name__)
    layer = layer._inner
layers.append(type(layer).__name__)
print(f"Stack: {' → '.join(layers)}")
print("\nAudit wraps Security wraps Retry wraps Adapter")

---
## 12. OTel Spans in Security Module

SecurityModule uses `_span()` from BaseModule to create child spans for each phase.

In [ ]:
print(inspect.getsource(SecurityModule.invoke))

print("\nSpan tree when OTel is enabled:")
print("  security")
print("    ├─ security.pii_redact_outbound")
print("    ├─ (inner.invoke)")
print("    ├─ security.pii_redact_inbound")
print("    └─ security.sign")

---
## 13. Exception Propagation

In [ ]:
inner_fail = MagicMock(spec=LLMProvider)
inner_fail.name = "test"
inner_fail.model_name = "test"
inner_fail.invoke = AsyncMock(side_effect=RuntimeError("provider down"))

mod_fail = SecurityModule({}, inner_fail)

try:
    await mod_fail.invoke([Message(role="user", content="hi")])
except RuntimeError as e:
    print(f"Exception propagated: {e}")
    print("SecurityModule never swallows errors.")

---
## 14. Registry Integration

In [ ]:
clear_cache()

# security=True uses config.toml defaults
model = load_model("anthropic", security=True)
print(f"Type: {type(model).__name__}")
print(f"PII: {model._pii_enabled}")
print(f"Signing: {model._signing_enabled}")

In [ ]:
clear_cache()

# Override: PII only, custom patterns, no signing
model = load_model("anthropic", security={
    "pii_enabled": True,
    "pii_custom_patterns": [{"name": "MRN", "pattern": r"MRN-\d{6}"}],
    "signing_enabled": False,
})
print(f"Type: {type(model).__name__}")
print(f"PII: {model._pii_enabled}")
print(f"Signing: {model._signing_enabled}")
print(f"Patterns: {len(model._pii_detector._patterns)} (5 builtin + 1 custom)")

---
## 15. Implementation Details

In [ ]:
print("=== SecurityModule.__init__ ===")
print(inspect.getsource(SecurityModule.__init__))

In [ ]:
print("=== SecurityModule._redact_messages ===")
print(inspect.getsource(SecurityModule._redact_messages))

In [ ]:
print("=== SecurityModule._attach_signature ===")
print(inspect.getsource(SecurityModule._attach_signature))

---
## Summary

| Component | What | Why |
|-----------|------|-----|
| `SecurityModule` | Per-invoke PII redaction + request signing | Federal compliance (NIST 800-53) |
| `RegexPiiDetector` | Built-in patterns: SSN, CC, email, phone, IPv4 | Works out of the box |
| `PiiDetector` protocol | Pluggable backend (Presidio, spaCy, custom) | Extensible for domain-specific PII |
| Custom patterns | `pii_custom_patterns` config | MRNs, badge numbers, etc. |
| `[PII:TYPE]` tags | Type-preserving redaction | Debugging without PII exposure |
| `HmacSigner` | HMAC-SHA256 (stdlib) | Zero-dep signing |
| `canonical_payload()` | Deterministic JSON bytes | Reproducible signatures |
| Bidirectional PII | Outbound (to LLM) + inbound (from LLM) | LLMs can echo back PII |
| OTel spans | `security`, `security.pii_*`, `security.sign` | Visible in trace waterfall |
| Stack position | Audit → Security → Retry | Audit sees redacted; retries are clean |

**Config**:
```python
load_model("anthropic", security=True)                          # PII + signing with defaults
load_model("anthropic", security={"signing_enabled": False})    # PII only
load_model("anthropic", security={"pii_enabled": False})        # Signing only
load_model("anthropic", security={"pii_custom_patterns": [...]}) # Custom patterns
load_model("anthropic", security=False)                         # Disable
```

**TOML defaults** (`config.toml`):
```toml
[modules.security]
enabled = false
pii_enabled = true
pii_detector = "regex"
pii_custom_patterns = []
signing_enabled = true
signing_algorithm = "hmac-sha256"
signing_key_env = "ARCLLM_SIGNING_KEY"
```